In [ ]:
# 25/12/29(월) 9:00

# Advanced RAG란 무엇인가

**RAG**는 LLM(대규모 언어 모델)이 답을 만들 때, **외부 지식(문서/DB/웹 등)을 검색(Retrieval)해서 근거(Context)로 붙인 뒤 생성(Generation)**하는 구조다. 그런데 기본 RAG(Naive RAG)는 실무에서 다음 문제가 있다.

- 검색이 "비슷한 것"은 찾는데 "정답 근거"를 제대로 못 찾는 경우
- 근거가 있어도 LLM이 **hallucination(환각)**를 섞거나, 근거와 다른 답을 하는 경우
- 검색한 문서가 너무 길고 복잡해서 **정답에 필요한 부분**(**chunk**)을 LLM이 제대로 찾지 못해 답변 품질이 안 좋은 경우

**Advanced RAG**: 이런 "현업형 문제"를 줄이기 위해 **데이터베이스 구축 단계, 검색 전단계, 검색 후 단계, 생성 단계에서 고도화**하는 설계 패턴.
기본 RAG(Naive RAG)가 "사람이 대충 찾아준 참고자료로 글 쓰는 것"이라면, Advanced RAG는 "사서(검색) + 편집자(정제) + 팩트체커(검증) + 작가(생성)가 협업하는 파이프라인"에 가깝다. 딱 하나의 기법이 아니라 추가 가능한 여러 기법. 

## 종류

### 검색 품질을 올리는 고급 Retrieval

- **하이브리드 검색**: 키워드(BM25) + 벡터(임베딩) 조합
- **멀티-쿼리/쿼리 재작성(Query rewriting)**: 질문을 더 잘 검색되게 바꿔 검색 성공률을 올림
- **메타데이터 필터링**: 메타데이터 필터링을 통해 범위를 좁혀 **정확도**를 올림. 의미기반 검색과 키워드 검색을 조합한 효과.
- **리랭킹(Re-ranking)**: 후보 문서를 많이 가져온 뒤, "정답에 가까운 순서"로 다시 정렬

### Chunking/인덱싱 전략 고도화
- 벡터 데이터 베이스 구축시 어떤 구조로 문서를 분할하고 Indexing 할지의 전략
  - **구조 기반 분할**: 헤더/섹션/표/코드블록
  - **계층형 인덱스**: 문서 요약 → 섹션 → 세부 chunk, Parent-Child 구조
  - **윈도우 확장**: 필요 시 앞 뒤 문맥을 함께 붙임
  
### 검색 결과의 "정제/조립" 단계 추가

- 중복 제거, 노이즈 제거, 관련 부분만 발췌
- 여러 chunk를 **질문 관점으로 재구성**
  → LLM에 그대로 던지는 게 아니라 "답변에 쓰기 좋은 근거 묶음"으로 편집하여 전달.

### 생성 단계의 신뢰성 강화(가드레일)

- **근거 기반 답변 강제**: 검색한 문서에 답변의 근거가 없으면 "답을 모른다."고 답변하도록 프롬프트를 구성.
- **인용/근거 스니펫 포함**: 어느 문서에서 답이 나왔는지 근거를 보여주도록 해 답변의 신뢰성을 높인다.
- **자기검증(Verification)**: 답을 만든 뒤 "근거와 모순 없는지" 재확인 하도록 한다.
- **불확실성 처리**: 애매하면 추가 질문(Clarification)을 하거나 또는 답변 범위 제한.

## Advanced RAG가 필요한 일반적인 경우

- 문서가 많고(수천~수백만), 구조가 복잡한 경우(매뉴얼/규정/기술문서)
- 질문이 단순 검색이 아니라 "비교/조건/절차/근거 요구"가 많은 복잡한 질문인 경우.
- 최신성/정확성이 중요한 경우(정책, 금융, 의료, 보안, 운영 장애 대응)
- "대충 그럴듯한 답"이 아니라 **근거가 있는 답**이 필수인 경우


In [1]:
###############################################################
# VectorStore, Retriever 준비
###############################################################
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain_text_splitters import MarkdownHeaderTextSplitter
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

from langchain_openai import ChatOpenAI
from langchain_qdrant import FastEmbedSparse, QdrantVectorStore, RetrievalMode
from qdrant_client import QdrantClient, models
from qdrant_client.models import Distance, SparseVectorParams, VectorParams
from langchain_openai import OpenAIEmbeddings

In [3]:
# 데이터 준비

def load_and_split_olympic_data(file_path="data/olympic_wiki.md"):
    with open(file_path, "r", encoding="utf-8") as fr:
        olympic_text = fr.read()

    # Split
    splitter = MarkdownHeaderTextSplitter(
        headers_to_split_on=[
            ("#", "Header 1"),
            ("##", "Header 2"),
            ("###", "Header 3"),
        ],
        # strip_headers=False, # 문서에 header 포함 여부(default: True - 제거)
    )

    return splitter.split_text(olympic_text)

In [4]:
#################################################################
# Vector DB 연결
# retriever 생성
#################################################################

# collection 삭제후 생성 (데이터 넣지는 않음)
def get_vectorstore(collection_name: str = "olympic_info_wiki"):

    #######################################
    # Qdrant Collection 생성 (sparse + dense)
    #######################################
    dense_embeddings = OpenAIEmbeddings(model="text-embedding-3-large")
    sparse_embeddings = FastEmbedSparse(model_name="Qdrant/bm25")

    client = QdrantClient(host="localhost", port=6333)

    #삭제후 생성
    if client.collection_exists(collection_name):
        result = client.delete_collection(collection_name=collection_name)

    client.create_collection(
        collection_name=collection_name,
        vectors_config={"dense": VectorParams(size=3072, distance=Distance.COSINE)},
        sparse_vectors_config={ 
            "sparse": SparseVectorParams()
        },
    )

    ######################################
    # VectorStore 생성 (Hybrid 모드)
    ######################################
    vector_store = QdrantVectorStore(
        client=client,
        collection_name=collection_name,
    
        embedding=dense_embeddings,
        
        sparse_embedding=sparse_embeddings,
        retrieval_mode=RetrievalMode.HYBRID,
    
        vector_name="dense",
        sparse_vector_name="sparse",
    )
    
    ######################################
    # Document들 추가
    ######################################
    documents = load_and_split_olympic_data()
    vector_store.add_documents(documents=documents)

    return vector_store


def get_retriever(vector_store, k: int = 5):
    retriever = vector_store.as_retriever(
        search_kwargs={"k": k}
    )
    return retriever

In [6]:
!uv pip install fastembed

Resolved 27 packages in 336ms                                        
⠙ Preparing packages... (0/4)                                                   
⠙ Preparing packages... (0/4)--------------     0 B/60.15 KiB           
⠙ Preparing packages... (0/4)-------------- 14.89 KiB/60.15 KiB         
⠙ Preparing packages... (0/4)-------------- 30.89 KiB/60.15 KiB         
⠙ Preparing packages... (0/4)-------------- 30.89 KiB/60.15 KiB         
mmh3                 ------------------------------     0 B/39.73 KiB
⠙ Preparing packages... (0/4)-------------- 30.89 KiB/60.15 KiB         
mmh3                 ------------------------------ 16.00 KiB/39.73 KiB
⠙ Preparing packages... (0/4)-------------- 30.89 KiB/60.15 KiB         
mmh3                 ------------------------------ 16.00 KiB/39.73 KiB
⠙ Preparing packages... (0/4)-------------- 30.89 KiB/60.15 KiB         
mmh3                 ------------------------------ 16.00 KiB/39.73 KiB
⠙ Preparing packages... (0/4)-------------- 30.89 Ki

In [8]:
vectorstore = get_vectorstore()
basic_retriever = get_retriever(vectorstore)  # naive rag

In [11]:
basic_retriever.invoke("근대 올림픽은 언제 시작되었나요?")

[Document(metadata={'Header 1': '올림픽', '_id': 'db7294c4-c7cd-4b9e-832b-165c74eb0d8f', '_collection_name': 'olympic_info_wiki'}, page_content='올림픽(영어: Olympic Games, 프랑스어: Jeux olympiques)은 전 세계 각 대륙 각국에서 모인 수천 명의 선수가 참가해 여름과 겨울에 스포츠 경기를 하는 국제적인 대회이다. 전 세계에서 가장 큰 지구촌 최대의 스포츠 축제인 올림픽은 세계에서 가장 인지도있는 국제 행사이다. 올림픽은 2년마다 하계 올림픽과 동계 올림픽이 번갈아 열리며, 국제 올림픽 위원회(IOC)가 감독하고 있다. 또한 오늘날의 올림픽은 기원전 8세기부터 서기 5세기에 이르기까지 고대 그리스 올림피아에서 열렸던 올림피아 제전에서 비롯되었다. 그리고 19세기 말에 피에르 드 쿠베르탱 남작이 고대 올림피아 제전에서 영감을 얻어, 근대 올림픽을 부활시켰다. 이를 위해 쿠베르탱 남작은 1894년에 IOC를 창설했으며, 2년 뒤인 1896년에 그리스 아테네에서 제 1회 올림픽이 열렸다. 이때부터 IOC는 올림픽 운동의 감독 기구가 되었으며, 조직과 활동은 올림픽 헌장을 따른다. 오늘날 전 세계 대부분의 국가에서 올림픽 메달은 매우 큰 영예이며, 특히 올림픽 금메달리스트는 국가 영웅급의 대우를 받으며 스포츠 스타가 된다. 국가별로 올림픽 메달리스트들에게 지급하는 포상금도 크다. 대부분의 인기있는 종목들이나 일상에서 쉽게 접하고 즐길 수 있는 생활스포츠 종목들이 올림픽이라는 한 대회에서 동시에 열리고, 전 세계 대부분의 국가 출신의 선수들이 참여하는 만큼 전 세계 스포츠 팬들이 가장 많이 시청하는 이벤트이다. 2008 베이징 올림픽의 모든 종목 누적 시청자 수만 47억 명에 달하며, 이는 인류 역사상 가장 많은 수의 인구가 시청한 이벤트였다.  \n또한 20세기에 올림픽 운동이 발전함에 따라, IOC는 변화하는 세계의 사회 환경에 적응해

# Rerank

## 개념

- RAG의 정확도는 관련 정보의 컨텍스트 내 존재 유무가 아니라 순서가 중요하다. 즉, 관련 정보가 컨텍스트 내 상위권에 위치하고 있을 때 좋은 답변을 얻을 수 있다는 뜻이다. 
- **Rerank**는 RAG 시스템에서 **초기 검색 단계에서 추출된 후보 문서들의 순위를 재조정**하는 기법이다. 
- 벡터 유사도 기반의 빠른 1차 검색 후, 보다 정밀한 모델(예: Cross-encoder, LLM 등)을 활용해 질문과 검색된 문서간의 의미론적 관련성을 평가하여, 실제로 답변 생성에 가장 **적합한 문서들이 상위**에 오도록 순서를 다시 매긴다. 이를 통해 LLM이 더 정확하고 관련성 높은 정보를 바탕으로 답변을 생성할 수 있게 도와준다.

## 방법

- **Cross-encoder 기반 Rerank**  
  - Cross Encoder를 이용해서 순위를 재 지정.
  - Cross-encoder
    - 질문과 문서를 같이 입력으로 받아 둘간의 유사도 점수를 예측하도록 학습한 모델.
    - 학습을 두 문장의 유사도록 예측하도록 학습하였기 때문에 단순 유사도 검사 보다 두 문장간의 의미적 관련성등을 이용해 유사도를 예측하기 때문에 더 정확한 결과.
  - 1차적으로 검색한 문서와 질문간의 유사도를 **cross-encoder**로 다시 계산해서 문서의 순위를 재 조정.
  
  > - **Bi-encoder**
  >     - 질문 (query)와 문서(document)를 각각 독립적으로 인코딩한 후, 벡터 간 유사도 계산
  >     - Encoder 모델은 개별 문장을 입력받아 embedding vector를 출력한다. 질문과 문서를 각각 encoding한 뒤에 둘 간의 유사도를 계산.
  
  ![bi_crosss_encoder](figures/bi_cross_encoder.png)

  \[출처:https://aws.amazon.com/ko/blogs/tech/korean-reranker-rag/\]

- **LLM 기반 Rerank**  
  GPT-3, GPT-4 등 LLM을 활용해 각 문서가 질문에 얼마나 부합하는지 평가하여 순위를 매긴다. 성능이 뛰어나지만 비용이 높고 응답 속도가 느릴 수 있다.
## Rerank RAG 프로세스  
1. 1차 검색(예: 임베딩 기반 벡터 검색)으로 상위 k개 문서 추출.  
2. Reranker 모델에 질문-문서 쌍을 입력.  
3. 각 쌍의 관련성 점수 산출 및 재정렬.  
4. 상위 n개 문서를 LLM의 컨텍스트로 전달하여 답변 생성.

## 장단점

- **장점**  
  - Rerank를 적용하면 단순 벡터 유사도 기반 검색보다 훨씬 정교하게 질문과 관련된 정보를 추출할 수 있다. 
  - 실제로 생성되는 답변의 품질이 크게 향상되며, 도메인 특화 정보나 복잡한 질의에도 높은 정확도를 보인다.

- **단점**  
  - Cross-encoder나 LLM 기반 Rerank는 연산량이 많아 실시간 응답이 필요한 대규모 서비스에선 속도 저하가 발생 가능.
  - 초기 검색 결과(상위 k개)에만 적용하므로, 1차 검색의 품질이 낮으면 Rerank 효과가 제한적일 수 있다.

## CrossEncoder Reranker 예제

# HyDE (Hypothetical Document Embedding)
## 개념
- 질문(query)에 대한 가상의 답변 문서를 생성하고, 이 생성된 가상의 답변 문서를 임베딩하여 검색에 활용하는 기법이다.
- 일반적인 RAG에서 검색은 질문을 임베딩하여 문서 임베딩과 직접 비교한다. 
- 질문("파리는 어떤 도시인가?")과 답변 문서("파리는 프랑스의 수도이며...")는 표현 방식이 다르다는 문제가 있다. 
- 즉 의미적으로는 관련있지만 벡터 공간(임베딩 벡터간의 유사서)에서는 거리가 멀 수가 있다.
- 그래서 HyDE는 질문과 문서가 아니라 질문으로 가상의 답변 문서를 만들고 **가상의 답변문서와 저장된 문서들간의 유사도**를 비교한다.

## HyDE 프로세스

1. 가상 문서 생성
   -  LLM을 사용해 질문에 대한 가상의 답변 문서 생성
   -  이때 성능이 좋은 LLM을 사용하는 것이 좋다.
2. 임베딩 변환
   - `1`에서 생성한 가상 문서를 벡터로 임베딩
3. 유사도 검색
   - 가상 문서 임베딩으로 Vector Store에 저장된 문서들 중 유사한 문서를 검색
4. 답변 생성
   - 검색된 실제 문서를 바탕으로 최종 답변 생성

## 장단점
- **장점**
  - 질문-문서 간 의미적 차이를 해결 해서 정확한 문서 검색 가능
  - 질문 표현 방식에 덜 민감
- **단점**
  - 가상 문서의 품질에 따른 성능 편차가 발생한다.
    - 가상 문서 생성 시 환각(hallucination) 위험
  - 추가적인 LLM 호출로 인한 비용 증가한다.

#  MultiQueryRetriever

## 개념
- 하나의 사용자 질문으로 **여러 개의 다양한 질문을 생성하여 검색**을 수행하는 방법이다.
- 단일 질문의 한계를 극복하고 다각도에서 관련 정보 검색할 수있다.
  - 기본 RAG는 사용자의 질문의 질(quality)에 따라 검색 결과가 좌우된다.
  - 사용자가 한 질문에만 의존하는 것이 아니라 그 질문을 바탕으로 **다양한 의미의 질문들을 생성해서 단일 질문이 가지는 표현의 한계를 보완**한다.
    - 동일한 질문을 다른 각도에서 접근할 수있다.
    - 다양한 어휘와 표현으로 질문을 재구성한다.
- 예)
  - **원본 질문**: "딥러닝의 장점은 무엇인가?"
  - **생성된 질문들**:
    1. "딥러닝이 전통적인 머신러닝보다 나은 점은?"
    2. "딥러닝을 사용하면 얻을 수 있는 이익은?"
    3. "딥러닝의 주요 강점과 특징은?"
    4. "딥러닝 기술의 핵심 우위는?"
## 실행 프로세스

1. 질문 생성   
   - LLM을 사용해 원본 질문을 3-5개의 서로 다른 질문으로 변환
2. 병렬 검색
   - 생성된 각 질문으로 독립적으로 문서 검색 수행
3. 결과 통합
   - 여러 검색 결과를 하나로 병합
4. 중복 제거
   - 동일한 문서가 여러 번 검색된 경우 중복 제거
5. 최종 답변
   - 통합된 문서 세트를 바탕으로 답변 생성

## 장단점
- **장점**
    - 단일 질문으로 놓칠 수 있는 관련 문서 발견 수 있다.
    - 사용자 질문 표현 방식의 한계 극복
    - 더 포괄적이고 완전한 정보 검색 및 수집을 할 수있다.
- **단점**
    - 여러 번의 LLM 호출과 검색 수행이 실행 되므로 **계산비용, 토큰비용, 응답시간이 증가한다.**
    - 생성된 질문의 품질에 따른 성능 편차가 있을 수 있다.
    - 생성된 질문에 따라 원래 질문과 관련성 낮은 문서도 검색될 수 있어 최종 답변을 방해하는 노이즈가 증가할 수있다.

# MapReduce RAG 방식

## 개요

- RAG(Retrieval-Augmented Generation)에서 검색된 문서들 중 질문과 관련성이 높은 문서만을 선별하여 더 정확한 답변을 생성하는 방법이다.
- 검색된 문서들 중에서 질문 답변에 실제로 도움이 되는 문서만을 LLM을 통해 선별한 후 전달하는 방식이다.

## MapReduce 방식 프로세스
1. Map (문서 검색)
   - 벡터스토어에서 질문과 유사한 문서들을 의미적 유사도 검색으로 찾는다.
   - 이 단계에서는 단순 벡터 유사도만 고려하므로 질문과 직접적인 관련이 없는 문서도 포함될 수 있다.
2. Reduce (문서 선별 및 요약)
   - 검색된 각 문서가 질문 답변에 실제로 도움이 되는지 LLM에게 평가 요청한다.
   - 관련성이 높은 문서들만 선별하여 요약하거나 결합한다.
   - 필요시 여러 문서의 정보를 통합하여 더 응답에 적합한 컨텍스트를 생성한다.
3.  Generate (최종 답변 생성)
    - 질문과 선별된 컨텍스트를 함께 LLM에 전달하여 최종 답변을 생성한다.

## 장단점

- **장점**
  - **높은 정확도**: 질문과 직접 관련된 정보만 사용하여 더 정확한 답변을 생성한다.
  - **노이즈 제거**: 유사하지만 관련 없는 정보로 인한 혼동을 방지한다.
  - **컨텍스트 최적화**: 제한된 토큰 범위 내에서 가장 유용한 정보만 전달한다.
  - **확장성**: 많은 문서가 검색되어도 중요한 정보만 선별하여 처리할 수 있다.
- 단점
  - **추가 비용**: 문서 선별을 위한 LLM 호출로 인한 비용이 증가한다.
  - **처리 시간**: 문서 평가 단계가 추가되어 응답 속도가 저하된다.
  - **복잡성**: 구현과 관리가 더 복잡하다.
  - **의존성**: 문서 선별 성능이 LLM의 판단 능력에 크게 의존한다.

# Self Query Retriever

- **Self-Query Retriever**는 사용자의 자연어 질문을 LLM을 통해 '의미 기반 검색 쿼리(semantic query)'와 '메타데이터 기반 필터 조건(metadata filter)'으로 자동 분해/구조화하여, 벡터 검색과 조건 기반 필터링을 동시에 수행하도록 설계된 Advanced RAG의 고급 Retriever 기법이다. 이를 통해 메타데이터의 속성을 정확히 반영한 정밀 검색이 가능하다.
- 즉 사용자 질의로 부터 메타데이터의 필터 조회시 사용할 값을 추출하여 metadata 조건 기반 필터링을 할 수 있도록 한다.

## Self Query Retriever의 구성 요소

- **LLM (Large Language Model)**
    - 자연어로 표현된 사용자 질문을 받아, 이를 메타데이터 조건과 필터링 조건이 포함된 정형 쿼리(Structured Query)로 변환하는 역할을 한다.
- **StructuredQuery**
    - 사용자의 질문을 기반으로 생성되는 구조화된 쿼리로 다음 두가지가 생성된다.
      - 벡터 유사도 검색을 위한 **의미 기반 검색 쿼리**(semantic query)
      - 문서 메타데이터를 이용한 **필터링 조건**(metadata filter)
  
- **Query Translator**
    - 생성된 StructuredQuery를 특정 벡터 데이터베이스(Qdrant, Chroma 등)의 쿼리 언어로 번역하여, 실제 검색이 가능하도록 한다.
- **Vector Database**
    - 변환된 쿼리를 바탕으로 벡터 유사도 검색과 메타데이터 조건 필터링을 함께 수행하여 관련 문서를 반환한다.

## Self Query Retriever 작동 원리

![selfquery retriever](figures/selfquery_retriever.jpg)

1. 사용자가 자연어 질의(Query)를 입력한다.
2. LLM이 입력된 자연어 질문을 해석하여 **Query Constructor**가 **StructuredQuery**로 변환한다.
3. **Query Translator**가 StructuredQuery를 **벡터 데이터베이스에서 이해할 수 있는 쿼리**로 변환한다.
4. 벡터 데이터베이스가 변환된 쿼리에 따라 문서를 검색한다.
5. 최종적으로, 검색 결과가 사용자에게 제공된다.

## 사용 예시
1. Query
    - "2023년에 발표된 OpenAI의 GPT 모델 관련 논문을 찾아줘."
2. Query Constructor가 위 질문을 다음과 같은 형태의 StructuredQuery로 변환한다.

    ```json
    {
        "query": "GPT 모델 논문",
        "filter": {
            "must": [
                {
                    "key": "year",
                    "match": {
                        "value": 2023
                    }
                },
                {
                    "key": "author",
                    "match": {
                        "value": "OpenAI"
                    }
                }
            ]
        }
    }
    ```
     - query: 벡터검색에 사용될 자연어 의미기반 쿼리(semantic query)
     - filter: 메타데이터 기반 필터 조건
3. 위의 StruncturedQuery는 Retriever와 연결된 Vector database의 검색 형식에 맞춰 query translator에 의해 변환 되고 이것을 이용해 검색을 진행한다. (형식은 DB 마다 다르다.)


## Self Query Retriever의 장점

- **정밀성**: 메타데이터 조건을 정확하게 지정하여 원하는 문서를 정밀하게 검색할 수 있다.
- **효율성**: 메타데이터 필터링을 통해 관련 없는 문서를 미리 제거하여 검색 대상 문서의 후보 집합을 사전에 축소함으로써, 벡터 유사도 계산 대상이 줄어들고 전체 검색 연산량을 감소시킬 수 있다.
- **사용자 편의성**: 사용자가 복잡한 쿼리 조건을 직접 작성하지 않고, 자연어로 간편하게 질문할 수 있다.

## 예제
- 필요 패키지 설치
  - pip install langchain langchain-community langchain-openai langchain-qdrant lark

> ### Lark 패키지
> 
> - Lark는 텍스트를 구조적으로 분석하기 위한 파싱 라이브러리로, 미리 정의한 문법에 따라 문장을 해석하여 정해진 형태(parse tree 또는 abstract syntax tree) 형태의 구조화된 결과를 생성한다.
> - 이 라이브러리는 컴파일러, 인터프리터, DSL(도메인 특화 언어), 쿼리 언어, 수식 해석기처럼 구조화된 입력을 처리해야 하는 다양한 프로그램에서 활용된다.
> - LangChain의 **SelfQueryRetriever**에서는 LLM이 생성한 쿼리 조건 문자열(output)을 석하여 메타데이터 필터 조건으로 변환하기 위해 Lark가 사용된다. Lark는 텍스트 형식의 조건을 분석하여 벡터 검색 시 사용되는 StructuredQuery의 filter(metadata filter) 구조로 변환해주는 역할을 담당한다.

In [ ]:
from langchain_qdrant import QdrantVectorStore
from langchain_core.documents import Document
from langchain_openai import OpenAIEmbeddings
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams
from dotenv import load_dotenv

load_dotenv()

In [ ]:
documents = [
    Document(
        page_content="A bunch of scientists bring back dinosaurs and mayhem breaks loose",
        metadata={"year": 1993, "rating": 7.7, "genre": "science fiction"},
    ),
    Document(
        page_content="Leo DiCaprio gets lost in a dream within a dream within a dream within a ...",
        metadata={"year": 2010, "director": "Christopher Nolan", "rating": 8.2},
    ),
    Document(
        page_content="A psychologist / detective gets lost in a series of dreams within dreams within dreams and Inception reused the idea",
        metadata={"year": 2006, "director": "Satoshi Kon", "rating": 8.6},
    ),
    Document(
        page_content="A bunch of normal-sized women are supremely wholesome and some men pine after them",
        metadata={"year": 2019, "director": "Greta Gerwig", "rating": 8.3},
    ),
    Document(
        page_content="Toys come alive and have a blast doing so",
        metadata={"year": 1995, "genre": "animated"},
    ),
    Document(
        page_content="Three men walk into the Zone, three men walk out of the Zone",
        metadata={
            "year": 1979,
            "director": "Andrei Tarkovsky",
            "genre": "thriller",
            "rating": 9.9,
        },
    ),
]

In [ ]:
# Vector Store 생성

COLLECTION_NAME = "example"
embedding_model = OpenAIEmbeddings(model="text-embedding-3-small")

client = QdrantClient(host="localhost", port=6333)
client.create_collection(
    collection_name=COLLECTION_NAME,
    vectors_config=VectorParams(size=1536, distance=Distance.COSINE)
)

vectorstore = QdrantVectorStore(
    client=client,
    collection_name="example",
    embedding=embedding_model
)
vectorstore.add_documents(documents=documents)